## Colab Prep

Execute the following code cells to whenever you open/restart the notebook in Google Colab.

In [ ]:
!pip install "polars[all]"

In [ ]:
!wget https://github.com/WSU-DataScience/dsci_325_module6_basic_data_management_in_python/raw/main/sample_data.zip

In [ ]:
!unzip ./sample_data.zip

# Advanced Filters

In this section, we will take a closer look at common filtering patterns.  Note that this list is based on the Common Filter Operations section of the [SQL Alchemy tutorial](https://docs.sqlalchemy.org/en/latest/orm/tutorial.html) from the SQL Alchemy documentation, which is copyright © by SQLAlchemy authors and contributors. SQLAlchemy and its documentation are licensed under the MIT license.

In [12]:
import polars as pl
pl.Config.with_columns_kwargs = True

import seaborn as sns
%matplotlib inline

## Common Filter Operators

Most filters consist of the following operations.

* Equals/not equals and other inequalities
* Like/ilike
* In/not in
* Is Null/is not null
* And/or


## How we will proceed

Let's look at how each of the operations is performed in polars.  We need a dataset that is ripe for filtering, so we will return to the super hero data set.  Who doesn't love a super hero?

In [70]:
heroes = (pl.read_csv('./sample_data/heroes_information.csv', null_values=['-', '-99.0', ''])
          .rename({'':'ID'})
         )
heroes.head()

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
0,"""A-Bomb""","""Male""","""yellow""","""Human""","""No Hair""",203.0,"""Marvel Comics""",null,"""good""",441.0
1,"""Abe Sapien""","""Male""","""blue""","""Icthyo Sapien""","""No Hair""",191.0,"""Dark Horse Com...","""blue""","""good""",65.0
2,"""Abin Sur""","""Male""","""blue""","""Ungaran""","""No Hair""",185.0,"""DC Comics""","""red""","""good""",90.0
3,"""Abomination""","""Male""","""green""","""Human / Radiat...","""No Hair""",203.0,"""Marvel Comics""",null,"""bad""",441.0
4,"""Abraxas""","""Male""","""blue""","""Cosmic Entity""","""Black""",null,"""Marvel Comics""",null,"""bad""",null


## Common Filter Operators - Equality and Inequality

In all three frameworks, equalities/inequalities are performed using the regular Python operators on column expressions.

#### Check equality using `==`

In [71]:
(heroes
 .filter(pl.col('Eye color') == 'blue')
 .head(2))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
1,"""Abe Sapien""","""Male""","""blue""","""Icthyo Sapien""","""No Hair""",191.0,"""Dark Horse Com...","""blue""","""good""",65.0
2,"""Abin Sur""","""Male""","""blue""","""Ungaran""","""No Hair""",185.0,"""DC Comics""","""red""","""good""",90.0


#### Check not equal using `!=`

In [72]:
(heroes
 .filter(pl.col('Eye color') != 'blue')
 .head(2))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
0,"""A-Bomb""","""Male""","""yellow""","""Human""","""No Hair""",203.0,"""Marvel Comics""",null,"""good""",441.0
3,"""Abomination""","""Male""","""green""","""Human / Radiat...","""No Hair""",203.0,"""Marvel Comics""",null,"""bad""",441.0


#### Other inequalities

In [73]:
(heroes
 .filter(pl.col('Height') > 200)
 .filter(pl.col('Weight') <= 440)
 .head(2))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
17,"""Alien""","""Male""",null,"""Xenomorph XX12...","""No Hair""",244.0,"""Dark Horse Com...","""black""","""bad""",169.0
19,"""Amazo""","""Male""","""red""","""Android""",null,257.0,"""DC Comics""",null,"""bad""",173.0


## Reminder - Referencing Constructed Column

Recall that we can also reference newly constructed columns by using `pl.col('name')`.

In [74]:
(heroes
 .with_columns(Weight_kg = pl.col('Weight')/2.2046)
 .filter(pl.col('Weight_kg') <= 200)
 .head(2))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight,Weight_kg
i64,str,str,str,str,str,f64,str,str,str,f64,f64
1,"""Abe Sapien""","""Male""","""blue""","""Icthyo Sapien""","""No Hair""",191.0,"""Dark Horse Com...","""blue""","""good""",65.0,29.483807
2,"""Abin Sur""","""Male""","""blue""","""Ungaran""","""No Hair""",185.0,"""DC Comics""","""red""","""good""",90.0,40.823732


## <font color="red"> Exercise 6.4.1 - The Super Hero Dating Game - Part 1</font>

Yesterday, you notice a singles add in the local paper that reads

> SWF looking for BESHM (blue-eyed super hero).  Must be tall (70+ inches).  Only interested in bad boys! Must list height (in inches) in reply!

Write a query in each framework to help find candidates for this personal add.

In [75]:
# Your solution here

## Common Filter Operations -  LIKE and ILIKE

`LIKE` and `ILIKE` are both SQL idioms that 

* is used to match string patterns
* Uses the `%` wildcard
    * like `*` in a regular expression
* `LIKE` is case-sensitive
* `ILIKE` is case-insensitive
    * Actual details are platform dependent

### `polars` lacks `LIKE`/`ILIKE`

Instead we use

* `str.starts_with`
* `str.ends_with`
* `str.contains`

#### `LIKE 'Super%'`

In [77]:
(heroes
 .filter(pl.col('name').str.starts_with('Super'))
 .head(2))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
643,"""Superboy""","""Male""","""blue""",null,"""Black""",170.0,"""DC Comics""",null,"""good""",68.0
644,"""Superboy-Prime...","""Male""","""blue""","""Kryptonian""","""Black / Blue""",180.0,"""DC Comics""",null,"""bad""",77.0


#### `LIKE '%boy'`

In [78]:
(heroes
 .filter(pl.col('name').str.ends_with('boy'))
 .head(2))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
142,"""Bumbleboy""","""Male""",null,null,null,null,"""Marvel Comics""",null,"""good""",null
321,"""Hellboy""","""Male""","""gold""","""Demon""","""Black""",259.0,"""Dark Horse Com...",null,"""good""",158.0


#### `LIKE '%boy%'`

In [79]:
(heroes
 .filter(pl.col('name').str.contains('boy'))
 .head(2))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
142,"""Bumbleboy""","""Male""",null,null,null,null,"""Marvel Comics""",null,"""good""",null
321,"""Hellboy""","""Male""","""gold""","""Demon""","""Black""",259.0,"""Dark Horse Com...",null,"""good""",158.0


#### `ILIKE` using `str.to_lowercase()`

In [80]:
(heroes
 .filter(pl.col('name').str.to_lowercase().str.contains('boy'))
 .head(3))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
46,"""Astro Boy""","""Male""","""brown""",null,"""Black""",null,null,null,"""good""",null
75,"""Beast Boy""","""Male""","""green""","""Human""","""Green""",173.0,"""DC Comics""","""green""","""good""",68.0
142,"""Bumbleboy""","""Male""",null,null,null,null,"""Marvel Comics""",null,"""good""",null


## Cry 'Havoc!,' and let slip the dogs of RegEx

Many `polars` string methods, like `contains` accept regular expressions.

In [84]:
(heroes
 .filter(pl.col('Publisher').str.contains('DC|Marvel'))
 .filter(pl.col('name').str.contains('\s[Bb]oy|\wboy'))
 .head()
)

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
75,"""Beast Boy""","""Male""","""green""","""Human""","""Green""",173.0,"""DC Comics""","""green""","""good""",68.0
142,"""Bumbleboy""","""Male""",null,null,null,null,"""Marvel Comics""",null,"""good""",null
183,"""Colossal Boy""","""Male""",null,null,null,null,"""DC Comics""",null,"""good""",null
643,"""Superboy""","""Male""","""blue""",null,"""Black""",170.0,"""DC Comics""",null,"""good""",68.0
644,"""Superboy-Prime...","""Male""","""blue""","""Kryptonian""","""Black / Blue""",180.0,"""DC Comics""",null,"""bad""",77.0


### Getting help on `polar` string methods.

Currently, it is difficult to find help on the various `polars` string methods.  More information can be obtains as follows.

In [60]:
from polars.internals.expr.string import ExprStringNameSpace
dir(ExprStringNameSpace) # list all the str functions

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessor',
 'concat',
 'contains',
 'count_match',
 'decode',
 'encode',
 'ends_with',
 'extract',
 'extract_all',
 'json_path_match',
 'lengths',
 'ljust',
 'lstrip',
 'n_chars',
 'replace',
 'replace_all',
 'rjust',
 'rstrip',
 'slice',
 'split',
 'split_exact',
 'splitn',
 'starts_with',
 'strip',
 'strptime',
 'to_lowercase',
 'to_uppercase',
 'zfill']

In [62]:
# Get help on one of the methods.
?ExprStringNameSpace.contains

## Common Filter Operations -  `IN` and `NOT IN`

`SQL` has `IN` and `NOT IN`, which are used to check if a value is in/not in a collection.

### Using  `IN`/`NOT IN` in `polars`

* `polars` uses the column `pl.col('name').is_in` method
* Prepend column expression with `~` for `NOT IN`

#### `IN` 

In [64]:
(heroes
 .filter(pl.col('Publisher').is_in(['DC Comics', 'Marvel Comics']))
 .head(2))

,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
0,"""A-Bomb""","""Male""","""yellow""","""Human""","""No Hair""",203.0,"""Marvel Comics""",null,"""good""",441.0
2,"""Abin Sur""","""Male""","""blue""","""Ungaran""","""No Hair""",185.0,"""DC Comics""","""red""","""good""",90.0


#### `NOT IN` 

In [68]:
(heroes
 .filter(~pl.col('Publisher').is_in(['DC Comics', 'Marvel Comics']))
 .head(2))

,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
1,"""Abe Sapien""","""Male""","""blue""","""Icthyo Sapien""","""No Hair""",191.0,"""Dark Horse Com...","""blue""","""good""",65.0
6,"""Adam Monroe""","""Male""","""blue""",null,"""Blond""",null,"""NBC - Heroes""",null,"""good""",null


## <font color="red"> Exercise 6.4.2 - The Super Hero Dating Game - Part 2</font>

Yesterday, you notice another singles add in the local paper, which reads

> SBiM looking for SyFy super hero (will also consider Star Wars, Star Trek, or NBC - Heroes).  Eye color must be either blue or brown and last name must start with either B or P.

Write a query in `dfply` to help find candidates for this personal add.

In [15]:
# Your solution here

## Common Filter Operators - IS NULL/IS NOT NULL

`SQL` has `IS NULL` and `IS NOT NULL`, which are used to check for missing values.

### Using  `IS NULL`/`IS NOT NULL` in `polars`

`polars` use the column `is_null`/`is_not_null` methods

#### `IS NULL`

In [91]:
(heroes
 .filter(pl.col('Skin color').is_null())
 .head(2))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
0,"""A-Bomb""","""Male""","""yellow""","""Human""","""No Hair""",203.0,"""Marvel Comics""",null,"""good""",441.0
3,"""Abomination""","""Male""","""green""","""Human / Radiat...","""No Hair""",203.0,"""Marvel Comics""",null,"""bad""",441.0


#### `IS NOT NULL`  using `is_not_null`

In [95]:
(heroes
 .filter(pl.col('Skin color').is_not_null())
 .head(2))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
1,"""Abe Sapien""","""Male""","""blue""","""Icthyo Sapien""","""No Hair""",191.0,"""Dark Horse Com...","""blue""","""good""",65.0
2,"""Abin Sur""","""Male""","""blue""","""Ungaran""","""No Hair""",185.0,"""DC Comics""","""red""","""good""",90.0


## Common Filter Operations - `AND`/ `OR`

`SQL` has `AND` and `OR`, which are used to check for missing values.

### Using  `AND`/`OR` in `polars`

 In `polars`, we use 
 
 * The `&` operator for AND
 * the `|` operator for OR
 

#### `AND` using `&`

In [98]:
(heroes
 .filter((pl.col('Hair color') == 'No Hair') & (pl.col('Eye color') == 'blue'))
 .head(2))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
1,"""Abe Sapien""","""Male""","""blue""","""Icthyo Sapien""","""No Hair""",191.0,"""Dark Horse Com...","""blue""","""good""",65.0
2,"""Abin Sur""","""Male""","""blue""","""Ungaran""","""No Hair""",185.0,"""DC Comics""","""red""","""good""",90.0


#### `AND` using multiple `filter` calls

In [109]:
(heroes
 .filter(pl.col('Hair color') == 'No Hair')
 .filter(pl.col('Eye color') == 'blue')
 .head(2))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
1,"""Abe Sapien""","""Male""","""blue""","""Icthyo Sapien""","""No Hair""",191.0,"""Dark Horse Com...","""blue""","""good""",65.0
2,"""Abin Sur""","""Male""","""blue""","""Ungaran""","""No Hair""",185.0,"""DC Comics""","""red""","""good""",90.0


#### `OR`

In [110]:
(heroes
 .filter((pl.col('Hair color') == 'No Hair') | (pl.col('Eye color') == 'blue'))
 .head(2))

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,f64
0,"""A-Bomb""","""Male""","""yellow""","""Human""","""No Hair""",203.0,"""Marvel Comics""",null,"""good""",441.0
1,"""Abe Sapien""","""Male""","""blue""","""Icthyo Sapien""","""No Hair""",191.0,"""Dark Horse Com...","""blue""","""good""",65.0


#### Important NOTE
You need to break the Python habit of using `and` and `or` (hard to do)

In [111]:
(heroes
 .filter((pl.col('Hair color') == 'No Hair') and (pl.col('Eye color') == 'blue'))
 .head(2))

ValueError: Since Expr are lazy, the truthiness of an Expr is ambiguous. Hint: use '&' or '|' to chain Expr together, not and/or.

## <font color="red"> Exercise 6.4.3 - The Super Hero Dating Game - Part 3</font>

Yesterday, you notice one more singles add in the local paper, which read

> W4A (Woman for Androgynous) looking for super hero.  Must be either God/Eternal/Cosmic Entity; or have no body hair.  Bad heroes need not reply.

Write a query in all three frameworks to help find candidates for this personal add.  You should complete each query with **exactly one filter_by/where**.

In [112]:
# Your solution here